In [16]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Data Preprocessing
class DriverActivityDataset(Dataset):
    def __init__(self, data_file, annotation_file):
        # Load the .pt file (ensure the file is a tensor, not a list)
        self.data = torch.load(data_file)  # Shape: (19071, 34)

        # Ensure the data is a tensor and convert to np.argmax
        self.data = torch.tensor(self.data)  # Convert list to tensor if it's not already
        self.data = torch.argmax(self.data, dim=1)  # Shape: (19071,) - now contains np.argmax labels

        # Load the CSV file with annotations
        self.annotations = pd.read_csv(annotation_file)
        self.annotations = self.annotations.iloc[:, 1].values  # Assuming annotations are in the second column

        # Remove dummy -1 annotations
        self.valid_data = []
        self.valid_annotations = []
        for i in range(len(self.annotations)):
            if self.annotations[i] != -1:  # Only keep valid annotations
                self.valid_data.append(self.data[i])
                self.valid_annotations.append(self.annotations[i])

        self.valid_data = torch.tensor(self.valid_data)
        self.valid_annotations = torch.tensor(self.valid_annotations)

    def __len__(self):
        return len(self.valid_data)

    def __getitem__(self, idx):
        return self.valid_data[idx], self.valid_annotations[idx]

class TestDataset(Dataset):
    def __init__(self, data_file, annotation_file):
        # Load the .pt file (ensure the file is a tensor, not a list)
        self.data = torch.load(data_file)  # Shape: (16655, 34)

        # Convert the data into the appropriate format (same as the train dataset)
        self.data = torch.tensor(self.data)
        self.data = torch.argmax(self.data, dim=1)  # Shape: (16655,) - now contains np.argmax labels

        # Load the CSV file with annotations
        self.annotations = pd.read_csv(annotation_file)
        self.annotations = self.annotations.iloc[:, 1].values  # Assuming annotations are in the second column

        # Remove dummy -1 annotations
        self.valid_data = []
        self.valid_annotations = []
        for i in range(len(self.annotations)):
            if self.annotations[i] != -1:  # Only keep valid annotations
                self.valid_data.append(self.data[i])
                self.valid_annotations.append(self.annotations[i])

        self.valid_data = torch.tensor(self.valid_data)
        self.valid_annotations = torch.tensor(self.valid_annotations)

    def __len__(self):
        return len(self.valid_data)

    def __getitem__(self, idx):
        return self.valid_data[idx], self.valid_annotations[idx]


# Model Architecture (LSTM)
class TemporalModel(nn.Module):
    def __init__(self):
        super(TemporalModel, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=128, num_layers=2, batch_first=True)
        self.fc = nn.Linear(128, 34)  # 34 actions, each action corresponds to a class

    def forward(self, x):
        # Ensure x has shape (batch_size, sequence_length, 1)
        x = x.unsqueeze(-1).float()  # Add a dummy dimension for input size (batch_size, sequence_length, 1)

        # Pass through LSTM
        lstm_out, _ = self.lstm(x)  # lstm_out will have shape (batch_size, sequence_length, hidden_size)

        # Check if lstm_out is 3D, it should be
        #print(f"LSTM output shape: {lstm_out.shape}")  # Debugging line

        # Only use the output of the last frame (time step)
        last_out = lstm_out[:, -1, :]  # Taking the last timestep output

        # Feed last frame's LSTM output to the fully connected layer
        output = self.fc(last_out)
        return output

# Load data
train_file = '/content/run1b_2018-05-29-14-02-47.kinect_color.pt'  # .pt file containing class probability arrays
train_annotations_file = '/content/new_data.csv'  # CSV file containing frame annotations

train_dataset = DriverActivityDataset(train_file, train_annotations_file)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

def train_model(model, train_loader, val_loader=None, num_epochs=100, learning_rate=0.001, patience=10):
    # Set the model to training mode
    model.train()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

    best_accuracy = 0.0
    epochs_without_improvement = 0  # Early stopping counter

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, (frames, annotations) in enumerate(train_loader):
            frames = frames.unsqueeze(-1).float()  # Reshape frames (batch_size, sequence_length, 1)
            annotations = annotations.long()  # Ensure annotations are in long format

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(frames)

            # Calculate the loss
            loss = criterion(outputs, annotations)

            # Backpropagation
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_train += annotations.size(0)
            correct_train += (predicted == annotations).sum().item()

        # Compute average loss and accuracy for the epoch
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = 100 * correct_train / total_train

        # Print statistics every epoch
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%")

        # Update learning rate with scheduler
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.7, verbose=True)

        # Early stopping: Check for improvement in accuracy on validation set
        if val_loader is not None:
            val_accuracy = evaluate_model(model, val_loader)
            print(f"Validation Accuracy: {val_accuracy:.2f}%")

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            if epochs_without_improvement >= patience:
                print(f"Early stopping: No improvement for {patience} epochs.")
                break



# Instantiate and train the model
model = TemporalModel()
train_model(model, train_loader)


def test_model(model, test_file, test_annotations_file):
    model.eval()  # Set the model to evaluation mode

    test_dataset = TestDataset(test_file, test_annotations_file)  # Use the new TestDataset class
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    total_correct = 0
    total_samples = 0

    with torch.no_grad():  # No gradients needed during inference
        for frames, annotations in test_loader:
            frames = frames.unsqueeze(-1).float()  # Reshape the frames as needed
            annotations = annotations.long()

            # Forward pass
            outputs = model(frames)

            # Get predictions (choose the class with the highest probability)
            _, predicted = torch.max(outputs, 1)

            # Calculate accuracy
            total_samples += annotations.size(0)
            total_correct += (predicted == annotations).sum().item()

    accuracy = total_correct / total_samples * 100
    print(f"Test Accuracy: {accuracy:.2f}%")


# Assuming test file is in .pt and .csv format
test_file = '/content/run1_2018-05-22-15-10-41.kinect_color.pt'
test_annotations_file = '/content/newtest.csv'
test_model(model, test_file, test_annotations_file)


<ipython-input-16-55b45ef5428f>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data = torch.load(data_file)  # Shape: (19071, 34)


Epoch [1/100], Loss: 2.4540, Training Accuracy: 37.06%


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [2/100], Loss: 1.9112, Training Accuracy: 48.04%
Epoch [3/100], Loss: 1.6649, Training Accuracy: 49.87%
Epoch [4/100], Loss: 1.4826, Training Accuracy: 57.51%
Epoch [5/100], Loss: 1.3357, Training Accuracy: 62.90%
Epoch [6/100], Loss: 1.2159, Training Accuracy: 70.72%
Epoch [7/100], Loss: 1.1299, Training Accuracy: 73.93%
Epoch [8/100], Loss: 1.0572, Training Accuracy: 76.08%
Epoch [9/100], Loss: 1.0008, Training Accuracy: 76.94%
Epoch [10/100], Loss: 0.9543, Training Accuracy: 78.49%
Epoch [11/100], Loss: 0.9174, Training Accuracy: 79.44%
Epoch [12/100], Loss: 0.8864, Training Accuracy: 80.34%
Epoch [13/100], Loss: 0.8619, Training Accuracy: 80.87%
Epoch [14/100], Loss: 0.8408, Training Accuracy: 81.18%
Epoch [15/100], Loss: 0.8243, Training Accuracy: 81.55%
Epoch [16/100], Loss: 0.8079, Training Accuracy: 81.68%
Epoch [17/100], Loss: 0.7938, Training Accuracy: 81.88%
Epoch [18/100], Loss: 0.7820, Training Accuracy: 81.99%
Epoch [19/100], Loss: 0.7719, Training Accuracy: 82.28%


<ipython-input-16-55b45ef5428f>:43: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data = torch.load(data_file)  # Shape: (16655, 34)


Test Accuracy: 44.10%


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class DriverActivityModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(DriverActivityModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(p=0.5)  # Dropout layer to prevent overfitting
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)  # Apply dropout after the activation
        x = self.fc2(x)
        return self.softmax(x)  # Softmax to get probability distribution over classes

# Load the data
data_file = '/content/run1b_2018-05-29-14-02-47.kinect_color.pt'
train_data = torch.load(data_file)  # Shape: (19071, 34) for training data
test_data_file = '/content/run1_2018-05-22-15-10-41.kinect_color.pt'
test_data = torch.load(test_data_file)  # Shape: (16655, 34) for test data

# Convert to tensors if necessary (if they are not already tensors)
train_data = torch.tensor(train_data)  # Ensure it's a tensor
test_data = torch.tensor(test_data)  # Ensure it's a tensor

# Prepare input and output tensors
X_train = train_data  # Input features (19071 x 34)
y_train = torch.argmax(X_train, dim=1)  # Convert class probabilities to class labels
X_test = test_data  # Test features (16655 x 34)
y_test = torch.argmax(X_test, dim=1)  # Convert test data class probabilities

# Create data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Hyperparameters
input_size = X_train.shape[1]  # 34 class probabilities
hidden_size = 128  # You can adjust this
output_size = 34  # Number of possible driver actions
learning_rate = 0.0001
num_epochs = 100
early_stopping_patience = 10  # Patience for early stopping
best_test_accuracy = 0.0
epochs_since_improvement = 0

# Initialize model, loss function, and optimizer
model = DriverActivityModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()  # Using cross entropy for classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.001)  # L2 regularization (weight_decay)

# Training loop with early stopping
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training phase
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs.float())  # Forward pass
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = (correct_train / total_train) * 100

    # Validation phase
    model.eval()
    correct_test = 0
    total_test = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.float())
            _, predicted = torch.max(outputs, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_accuracy = (correct_test / total_test) * 100

    # Print training and validation results
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, '
          f'Training Accuracy: {train_accuracy:.2f}%, '
          f'Test Accuracy: {test_accuracy:.2f}%')

    # Check if early stopping condition is met
    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        epochs_since_improvement = 0  # Reset the patience counter
    else:
        epochs_since_improvement += 1

    if epochs_since_improvement >= early_stopping_patience:
        print(f"Early stopping triggered after {epoch+1} epochs.")
        break

# Final test accuracy
print(f'Final Test Accuracy: {best_test_accuracy:.2f}%')


<ipython-input-19-0f81425410d5>:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(data_file)  # Shape: (19071, 34) for training data
<ipython-input-1

Epoch [1/100], Loss: 3.2043, Training Accuracy: 47.31%, Test Accuracy: 75.35%
Epoch [2/100], Loss: 2.9332, Training Accuracy: 66.30%, Test Accuracy: 81.00%
Epoch [3/100], Loss: 2.9141, Training Accuracy: 67.07%, Test Accuracy: 81.00%
Epoch [4/100], Loss: 2.9105, Training Accuracy: 67.25%, Test Accuracy: 81.75%
Epoch [5/100], Loss: 2.8926, Training Accuracy: 69.55%, Test Accuracy: 82.03%
Epoch [6/100], Loss: 2.8878, Training Accuracy: 69.77%, Test Accuracy: 82.04%
Epoch [7/100], Loss: 2.8877, Training Accuracy: 69.67%, Test Accuracy: 82.06%
Epoch [8/100], Loss: 2.8869, Training Accuracy: 69.83%, Test Accuracy: 82.06%
Epoch [9/100], Loss: 2.8867, Training Accuracy: 69.84%, Test Accuracy: 82.04%
Epoch [10/100], Loss: 2.8866, Training Accuracy: 69.87%, Test Accuracy: 82.06%
Epoch [11/100], Loss: 2.8863, Training Accuracy: 69.88%, Test Accuracy: 82.00%
Epoch [12/100], Loss: 2.8861, Training Accuracy: 69.81%, Test Accuracy: 82.05%
Epoch [13/100], Loss: 2.8858, Training Accuracy: 69.90%, Test

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Load the data
data_file = '/content/run1b_2018-05-29-14-02-47.kinect_color.pt'
data = torch.load(data_file)  # Shape: (19071, 34) for training data
test_data_file = '/content/run1_2018-05-22-15-10-41.kinect_color.pt'
test_data = torch.load(test_data_file)  # Shape: (16655, 34) for test data

# Convert loaded data (list) to tensors
X_train = torch.tensor(data).float()  # Ensure X_train is a tensor
y_train = torch.argmax(X_train, dim=1)  # Convert class probabilities to class labels
X_test = torch.tensor(test_data).float()  # Ensure X_test is a tensor
y_test = torch.argmax(X_test, dim=1)  # Convert test data class probabilities to labels

# Create DataLoader for batch processing
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the model architecture with a combination of TCN and RNN layers
class TemporalDriverActivityModel(nn.Module):
    def __init__(self, input_size=34, hidden_size=128, output_size=34, dropout_prob=0.5):
        super(TemporalDriverActivityModel, self).__init__()

        # Temporal Convolutional Network (TCN) block
        self.tcn = nn.Conv1d(input_size, hidden_size, kernel_size=3, padding=1)

        # Recurrent layer (LSTM)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, output_size)

        # Dropout layer for regularization
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        # Reshape input for TCN: (batch_size, channels, sequence_length)
        # Assuming sequence_length = 1 since your data doesn't seem to have a temporal sequence
        x = x.unsqueeze(2)  # Add a singleton dimension to make it (batch_size, input_size, 1)

        # Apply Temporal Convolution (TCN)
        x = torch.relu(self.tcn(x))  # Shape becomes (batch_size, hidden_size, 1)

        # Apply LSTM for temporal modeling
        x, _ = self.rnn(x.transpose(1, 2))  # LSTM expects (batch_size, seq_len, input_size)

        # Take the last output from LSTM
        x = x[:, -1, :]

        # Apply fully connected layer
        x = self.fc(x)

        # Apply dropout
        x = self.dropout(x)

        return x

# Initialize the model, loss function, and optimizer
model = TemporalDriverActivityModel(input_size=34, hidden_size=128, output_size=34, dropout_prob=0.5)
criterion = nn.CrossEntropyLoss()  # For classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model with early stopping
num_epochs = 100
best_test_acc = 0
early_stop_counter = 0
early_stop_patience = 10  # Number of epochs without improvement to stop training

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        correct_train += (predicted == labels).sum().item()
        total_train += labels.size(0)

    # Calculate training accuracy
    train_accuracy = (correct_train / total_train) * 100

    # Evaluate on test data
    model.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_test += (predicted == labels).sum().item()
            total_test += labels.size(0)

    # Calculate test accuracy
    test_accuracy = (correct_test / total_test) * 100

    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, "
          f"Training Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%")

    # Early stopping mechanism
    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy
        early_stop_counter = 0
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stop_patience:
        print("Early stopping triggered.")
        break

print(f"Final Test Accuracy: {best_test_acc:.2f}%")


<ipython-input-22-305870e2ea44>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(data_file)  # Shape: (19071, 34) for training data
<ipython-input-22-30587

Epoch [1/100], Loss: 2.1730, Training Accuracy: 38.92%, Test Accuracy: 93.65%
Epoch [2/100], Loss: 1.5297, Training Accuracy: 49.90%, Test Accuracy: 96.48%
Epoch [3/100], Loss: 1.4903, Training Accuracy: 50.11%, Test Accuracy: 97.21%
Epoch [4/100], Loss: 1.4714, Training Accuracy: 50.50%, Test Accuracy: 97.31%
Epoch [5/100], Loss: 1.4646, Training Accuracy: 50.48%, Test Accuracy: 97.50%
Epoch [6/100], Loss: 1.4497, Training Accuracy: 50.82%, Test Accuracy: 97.73%
Epoch [7/100], Loss: 1.4567, Training Accuracy: 50.46%, Test Accuracy: 97.58%
Epoch [8/100], Loss: 1.4547, Training Accuracy: 50.45%, Test Accuracy: 97.70%
Epoch [9/100], Loss: 1.4416, Training Accuracy: 50.92%, Test Accuracy: 97.84%
Epoch [10/100], Loss: 1.4539, Training Accuracy: 50.38%, Test Accuracy: 97.63%
Epoch [11/100], Loss: 1.4398, Training Accuracy: 50.86%, Test Accuracy: 97.74%
Epoch [12/100], Loss: 1.4342, Training Accuracy: 50.99%, Test Accuracy: 97.58%
Epoch [13/100], Loss: 1.4611, Training Accuracy: 50.12%, Test